In [54]:
import os
import json
import pandas as pd
import itertools
from typing import List, Dict, Any
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.schema import Document
from langchain_qdrant import QdrantVectorStore
from dotenv import load_dotenv

In [55]:
# Diretórios principais
BASE_DIR = BASE_DIR = os.getcwd()
DIR_PAI = os.path.dirname(BASE_DIR)  # Diretório pai
DIR_SRC = os.path.join(DIR_PAI,'src')  # Diretório src
DIR_VECTORSTORE = os.path.join(DIR_SRC,'vector_store') 
DIR_DATA = os.path.join(DIR_PAI, "data")  # Diretório de dados
DIR_DATA_RAW = os.path.join(DIR_DATA, "raw")  # Diretório para dados brutos
DIR_LOGS =  os.path.join(DIR_DATA, "logs") # Diretório de logs
DIR_DATA_REFINEMENT = os.path.join(DIR_DATA, "outputs_vision_and_extractor")  # Diretório de refinamento

In [56]:
class CollectionCreator:
    def __init__(self, config_file: str):
        self.config = self.load_config(config_file)
        load_dotenv()
        self.qdrant_url = os.getenv("QDRANT_URL")
        self.qdrant_api_key = os.getenv("QDRANT_API_KEY")

        if not self.qdrant_url or not self.qdrant_api_key:
            raise ValueError("Variáveis QDRANT_URL ou QDRANT_API_KEY não encontradas!")

        # Carrega os metadados do CSV
        self.file_to_metadata = self.load_metadata_from_csv()

    def load_config(self, config_file: str) -> Dict[str, Any]:
        """Carrega o arquivo de configuração JSON."""
        with open(config_file, 'r', encoding='utf-8') as f:
            return json.load(f)

    def load_metadata_from_csv(self) -> Dict[str, Dict[str, Any]]:
        """Carrega o CSV e cria um mapeamento arquivo_id -> metadados."""
        csv_path = os.path.join(DIR_DATA, "subcategoria_name.csv")
        df = pd.read_csv(csv_path)
        metadata_dict = df.set_index("arquivo_id").to_dict(orient="index")
        print("Metadados carregados do CSV:", metadata_dict)
        return metadata_dict

    def load_json_documents(self, file_id: str) -> List[Document]:
        """Carrega arquivos JSON, adiciona metadados do CSV e páginas."""
        dir_path = os.path.join(self.config['pdf_dir'], file_id)

        if not os.path.isdir(dir_path):
            print(f"Diretório {dir_path} não existe. Pulando.")
            return []

        documents = []
        for fname in os.listdir(dir_path):
            if fname.endswith("_resultado.json"):
                fpath = os.path.join(dir_path, fname)
                with open(fpath, 'r', encoding='utf-8') as f:
                    data = json.load(f)

                # Extração de informações
                text = data.get("unified_analysis", "")
                page_number = data.get("page", None)
                metadata = {
                    "source": fname,
                    "pag": page_number
                }

                # Adiciona metadados do CSV
                try:
                    arquivo_id = int(fname.split("_")[1])  # Extrai arquivo_id do nome do arquivo
                    metadata["arquivo_id"] = arquivo_id
                    if arquivo_id in self.file_to_metadata:
                        csv_metadata = self.file_to_metadata[arquivo_id]
                        metadata.update(csv_metadata)
                    else:
                        print(f"Nenhum metadado encontrado para arquivo_id {arquivo_id}")
                except (IndexError, ValueError):
                    print(f"Erro ao extrair arquivo_id de {fname}")

                print("Metadados finais para o documento:", metadata)
                documents.append(Document(page_content=text, metadata=metadata))

        return documents

    def clean_metadata(self, metadata: dict) -> dict:
        """Garante que o metadata seja serializável e limpo."""
        return {k: v for k, v in metadata.items() if isinstance(v, (str, int, float, bool, list, dict))}

    def create_collection(self, collection_config: Dict[str, Any]):
        embeddings_model = collection_config['embeddings_model']
        local_embeddings = HuggingFaceEmbeddings(model_name=embeddings_model, model_kwargs={'trust_remote_code': True})

        documents = collection_config['documents']
        chunk_size = collection_config['chunk_size']
        chunk_overlap = collection_config['chunk_overlap']

        # Se chunk_size == "Page", não faz chunking adicional
        if chunk_size == "Page":
            splits = documents
        else:
            # chunk_size deve ser inteiro
            if isinstance(chunk_size, str):
                raise ValueError("chunk_size deve ser 'Page' ou um inteiro.")
            if chunk_overlap is None:
                chunk_overlap = 0
            text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
            splits = text_splitter.split_documents(documents)

        for doc in splits:
            doc.metadata = self.clean_metadata(doc.metadata)

        QdrantVectorStore.from_documents(
            documents=splits,
            embedding=local_embeddings,
            url=self.qdrant_url,
            api_key=self.qdrant_api_key,
            collection_name=collection_config['collection_name'],
            force_recreate=True
        )

        print(f"Collection {collection_config['collection_name']} criada com sucesso.")

    def generate_collection_configs(self, documents_dict: Dict[str, List[Document]]) -> List[Dict[str, Any]]:
        embeddings_models = self.config['embeddings_models']
        chunk_sizes = self.config['chunk_sizes']
        chunk_overlaps = self.config['chunk_overlaps']

        all_docs = []
        for docs in documents_dict.values():
            all_docs.extend(docs)

        combos = itertools.product(embeddings_models, chunk_sizes, chunk_overlaps)
        configs = []
        for emb_model, c_size, c_overlap in combos:
            chunk_display = c_size if c_size != "Page" else "by-page"
            collection_name = f"{self.config['collection_name']}_chunk{chunk_display}_overlap{c_overlap}_{emb_model.split('/')[-1]}"
            extraction_config = {
                "collection_name": collection_name,
                "chunk_size": c_size,
                "chunk_overlap": c_overlap,
                "embeddings_model": emb_model,
                "documents": all_docs,
            }
            configs.append(extraction_config)

        return configs

    def create_collections(self):
        documents_dict = {}
        for arquivo_id in self.config['arquivo_ids_to_process']:
            if not arquivo_id.startswith("fluidos_"):
                continue
            docs = self.load_json_documents(arquivo_id)
            if docs:
                documents_dict[arquivo_id] = docs

        self.collection_configs = self.generate_collection_configs(documents_dict)

        for collection_config in self.collection_configs:
            self.create_collection(collection_config)

In [57]:
if __name__ == "__main__":
    config_file = os.path.join(DIR_VECTORSTORE, "config.json")
    processor = CollectionCreator(config_file)
    processor.create_collections()

Metadados carregados do CSV: {11484: {'subcategoria_nome': 'fluidos'}, 11640: {'subcategoria_nome': 'fluidos'}, 13271: {'subcategoria_nome': 'fluidos'}, 13417: {'subcategoria_nome': 'fluidos'}, 13472: {'subcategoria_nome': 'fluidos'}, 13572: {'subcategoria_nome': 'fluidos'}, 13852: {'subcategoria_nome': 'fluidos'}, 4802: {'subcategoria_nome': 'fluidos'}}
Metadados finais para o documento: {'source': 'fluidos_13472_pag10_resultado.json', 'pag': 10, 'arquivo_id': 13472, 'subcategoria_nome': 'fluidos'}
Metadados finais para o documento: {'source': 'fluidos_13472_pag11_resultado.json', 'pag': 11, 'arquivo_id': 13472, 'subcategoria_nome': 'fluidos'}
Metadados finais para o documento: {'source': 'fluidos_13472_pag12_resultado.json', 'pag': 12, 'arquivo_id': 13472, 'subcategoria_nome': 'fluidos'}
Metadados finais para o documento: {'source': 'fluidos_13472_pag13_resultado.json', 'pag': 13, 'arquivo_id': 13472, 'subcategoria_nome': 'fluidos'}
Metadados finais para o documento: {'source': 'flui

### Criação de índices

In [50]:
from qdrant_client import QdrantClient
from qdrant_client.http import models

# Carregando variáveis de ambiente
QDRANT_URL = os.getenv("QDRANT_URL")
QDRANT_API_KEY = os.getenv("QDRANT_API_KEY")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
COLLECTION_NAME = "fluidos_chunkby-page_overlap0_multilingual-e5-large"          # Substitua pelo nome da sua coleção
FIELD_NAME = "metadata.arquivo_id"                                                 # Campo a ser indexado

# Conexão com o Qdrant com autenticação
client = QdrantClient(
    url=QDRANT_URL,
    api_key=QDRANT_API_KEY  # Adicione a chave de API aqui
)

In [51]:
# Função para criar o índice no campo especificado
def create_index_for_field(collection_name, field_name):
    try:
        client.create_payload_index(
            collection_name=collection_name,
            field_name=field_name,
            field_schema=models.PayloadSchemaType.KEYWORD,  # Índice do tipo "keyword"
        )
        print(f"Índice criado com sucesso para o campo '{field_name}' na coleção '{collection_name}'.")
    except Exception as e:
        print(f"Erro ao criar índice: {e}")

# Verificação da configuração da coleção
def verify_index(collection_name):
    try:
        collection_info = client.get_collection(collection_name)
        print("Configuração atual da coleção:")
        print(collection_info.model_dump())
    except Exception as e:
        print(f"Erro ao verificar configuração da coleção: {e}")

# Executa a criação do índice e verifica a configuração
if __name__ == "__main__":
    create_index_for_field(COLLECTION_NAME, FIELD_NAME)
    verify_index(COLLECTION_NAME)

Índice criado com sucesso para o campo 'metadata.arquivo_id' na coleção 'fluidos_chunkby-page_overlap0_multilingual-e5-large'.
Configuração atual da coleção:
{'status': <CollectionStatus.GREEN: 'green'>, 'optimizer_status': <OptimizersStatusOneOf.OK: 'ok'>, 'vectors_count': None, 'indexed_vectors_count': 0, 'points_count': 156, 'segments_count': 2, 'config': {'params': {'vectors': {'': {'size': 1024, 'distance': <Distance.COSINE: 'Cosine'>, 'hnsw_config': None, 'quantization_config': None, 'on_disk': None, 'datatype': None, 'multivector_config': None}}, 'shard_number': 1, 'sharding_method': None, 'replication_factor': 1, 'write_consistency_factor': 1, 'read_fan_out_factor': None, 'on_disk_payload': True, 'sparse_vectors': {}}, 'hnsw_config': {'m': 16, 'ef_construct': 100, 'full_scan_threshold': 10000, 'max_indexing_threads': 0, 'on_disk': False, 'payload_m': None}, 'optimizer_config': {'deleted_threshold': 0.2, 'vacuum_min_vector_number': 1000, 'default_segment_number': 0, 'max_segment

In [40]:
# Filtrar pontos com arquivo_id igual a 13472
result = client.search(
    collection_name="fluidos_chunkby-page_overlap0_multilingual-e5-large",
    query_vector=[0] * 1024,  # Vetor de consulta (exemplo: vetor nulo)
    limit=10,  # Limite de resultados
    query_filter=models.Filter(  # Atualizado para usar "query_filter"
        must=[
            models.FieldCondition(
                key="arquivo_id",
                match=models.MatchValue(value=13472)  # Valor a ser filtrado
            )
        ]
    )
)

# Exibir os resultados da consulta
for point in result:
    print(f"ID: {point.id}, Metadata: {point.payload}")

In [41]:
result

[]

In [21]:
collection_info = client.get_collection("fluidos_chunkby-page_overlap0_multilingual-e5-large")
print(collection_info.model_dump())

{'status': <CollectionStatus.GREEN: 'green'>, 'optimizer_status': <OptimizersStatusOneOf.OK: 'ok'>, 'vectors_count': None, 'indexed_vectors_count': 0, 'points_count': 156, 'segments_count': 2, 'config': {'params': {'vectors': {'size': 1024, 'distance': <Distance.COSINE: 'Cosine'>, 'hnsw_config': None, 'quantization_config': None, 'on_disk': None, 'datatype': None, 'multivector_config': None}, 'shard_number': 1, 'sharding_method': None, 'replication_factor': 1, 'write_consistency_factor': 1, 'read_fan_out_factor': None, 'on_disk_payload': True, 'sparse_vectors': None}, 'hnsw_config': {'m': 16, 'ef_construct': 100, 'full_scan_threshold': 10000, 'max_indexing_threads': 0, 'on_disk': False, 'payload_m': None}, 'optimizer_config': {'deleted_threshold': 0.2, 'vacuum_min_vector_number': 1000, 'default_segment_number': 0, 'max_segment_size': None, 'memmap_threshold': None, 'indexing_threshold': 20000, 'flush_interval_sec': 5, 'max_optimization_threads': None}, 'wal_config': {'wal_capacity_mb':

In [14]:
points = client.scroll(collection_name="fluidos_chunkby-page_overlap0_multilingual-e5-large", limit=10, )
for point in points[0]:
    print(point.payload)

{'page_content': '# MANUAL TÉCNICO - TRANSMISSÃO AUTOMÁTICA (AC60E)\n\n## FLUIDOS, ADITIVOS E FILTROS - CAPACIDADES E ESPECIFICAÇÕES\n\n### Procedimentos\n\n---\n\n### 1. Procedimento de Drenagem\n\n1. **Medida do Fluido Drenado**\n   - Utilize um objeto graduado com capacidade maior que **9,5 Litros** para medir a quantidade de fluido drenado.\n\n2. **Remoção do Bujão de Drenagem**\n   - Remova o bujão de drenagem e aguarde até que o óleo pare totalmente de escorrer.\n\n3. **Reinstalação do Bujão de Drenagem**\n   - Após a remoção do óleo da transmissão, reinstale o bujão de drenagem com uma nova junta, aplicando um torque de **20 N.m**.\n\n4. **Abastecimento**\n   - Para reabastecer a transmissão com um novo óleo, efetue o procedimento de abastecimento.\n\n---\n\n### 2. Procedimento de Abastecimento\n\n1. **Remoção dos Bujões**\n   - Remova o bujão de abastecimento e o bujão de verificação.\n\n2. **Adição do Fluido**\n   - Adicione através do orifício de abastecimento a quantidade de

In [58]:
result = client.scroll(
    collection_name="fluidos_chunkby-page_overlap0_multilingual-e5-large",
    scroll_filter=models.Filter(  # Substituir "filter" por "scroll_filter"
        must=[
            models.FieldCondition(
                key="metadata.arquivo_id",
                match=models.MatchValue(value=13472)
            )
        ]
    ),
    limit=10
)
print(result)

([Record(id='03cff0d9-06eb-4862-876b-4bf3504df162', payload={'page_content': '# MANUAL DOUTOR-IE: FLUIDOS, ADITIVOS E FILTROS - CAPACIDADES E ESPECIFICAÇÕES\n\n## Seção: EMBREAGEM\n\n### 1. FLUIDO\n\n#### 1.1 ÓLEO DA EMBREAGEM\n- **Capacidade:**\n  - O nível de óleo da embreagem deve estar sempre entre as marcas "MIN" e "MAX" presentes no reservatório de óleo da embreagem.\n  \n- **Especificação:**\n  - **Tipo:** DOT 4', 'metadata': {'source': 'fluidos_13472_pag3_resultado.json', 'pag': 3, 'arquivo_id': 13472, 'subcategoria_nome': 'fluidos'}}, vector=None, shard_key=None, order_value=None), Record(id='08baf8ff-683e-46b3-8532-65432bd58cbc', payload={'page_content': '# Manual Doutor-IE: Transmissão Automática (V5A5A - 5 Marchas)\n\n## Fluido\n\n### Óleo da Transmissão Automática\n- **Capacidade:** Aproximadamente 9,7 Litros\n- **Especificação:** ATF SP III\n- **Tipo de Óleo:** Sintético\n\n## Pontos de Drenagem e Abastecimento\n- **Tubo de Abastecimento/Verificação**\n- **Tubo de Retorno

In [52]:
from qdrant_client.http import models

result = client.search(
    collection_name="fluidos_chunkby-page_overlap0_multilingual-e5-large",
    query_vector=[0] * 1024,  # Vetor de consulta nulo (exemplo)
    limit=10,
    query_filter=models.Filter(
        must=[
            models.FieldCondition(
                key="arquivo_id",
                match=models.MatchValue(value=13472)
            )
        ]
    )
)

if result:
    for point in result:
        print(f"ID: {point.id}, Metadata: {point.payload}")
else:
    print("Nenhum ponto encontrado com o arquivo_id especificado.")

Nenhum ponto encontrado com o arquivo_id especificado.


In [35]:
points = client.scroll(
    collection_name="fluidos_chunkby-page_overlap0_multilingual-e5-large",
    scroll_filter=models.Filter(),  # Sem filtro para retornar todos os pontos
    limit=3
)

for point in points[0]:  # points[0] contém os resultados
    print(f"ID: {point.id}, Metadata: {point.payload}")


ID: 00cd91fe-4eb0-4567-86d8-eff318211cdb, Metadata: {'page_content': '# MANUAL DOUTOR-IE: FLUIDOS, ADITIVOS E FILTROS – CAPACIDADES E ESPECIFICAÇÕES\n\n## DIREÇÃO\n\n### FLUIDO\n\n#### Óleo da Direção Hidráulica\n\n- **Capacidade:**\n  - **Motor Frio:**\n    - Temperatura do óleo: 20°C\n    - Nível de óleo deve estar entre as marcas "MIN" e "MAX" na haste de medição.\n  - **Motor Quente:**\n    - Temperatura do óleo: 80°C\n    - Nível de óleo deve estar entre as marcas "MIN" e "MAX" na haste de medição.\n\n- **Especificação:**\n  - G 004 000\n\n### Observações\n\n- Este modelo de veículo possui dois tipos diferentes de direção assistida: elétrica ou hidráulica.\n- O reservatório de fluido da direção encontra-se próximo à torre do amortecedor dianteiro esquerdo.\n\n### Localização do Reservatório no Veículo\n\n- O reservatório de fluido para direção está localizado próximo à torre do amortecedor dianteiro esquerdo. (Imagem ilustrativa disponível no manual).\n\n---\n\nEste documento unif

In [59]:
models.Filter(must=[models.FieldCondition(key="metadata.arquivo_id", match=models.MatchAny(any=[13472, 13472]))])

Filter(should=None, min_should=None, must=[FieldCondition(key='metadata.arquivo_id', match=MatchAny(any=[13472, 13472]), range=None, geo_bounding_box=None, geo_radius=None, geo_polygon=None, values_count=None)], must_not=None)